# Final Report on Predictive Modeling for Optimal Capital Market Selection

This is the final report notebook for Team 12 in FIN 377. In this notebook we will explore our processes, modeling, as well as our conclusions.

To start: These are all the packages we used throughout the project.

In [5]:
# Data download and cleaning packages.
import pandas as pd
import os
import zipfile
import numpy as np
# We tried to fuzzymatch firms to their names, but ran into trouble and manually matched the CUSIPS
# !pip install fuzzywuzzy
from fuzzywuzzy import fuzz, process

# Modeling packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, r2_score, precision_score, f1_score, recall_score, confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

# Other Packages
from tqdm import tqdm
# We did not end up using wrds in our final process, but we thought it was a cool database and included it.
# !pip install wrds
# import wrds

### On to the data!

We used a lot of data in this project. It was very hard to clean, but we got there in the end.

The goal was to end up with a merged dataset of 3 inputs:
- Jay Ritter's SPAC data
    - This data contains all SPAC mergers from 2016-2021.
- Jay Ritter's IPO data
    - This data contains all IPOs since 1975
- CCM data
    - This data contains 950 columns of observations on firms from 2000-2018

We tried several methods to refine our final dataset:

1. We started by merging the different CUSIPs provided in each dataset.
- The problem with this method was that we could merge all the data together, only to find that there were 0 SPAC observations ·in the resulting dataset.
- We learned this is because the CUSIPs overlapped a bit between SPAC data and both IPO and CCM data, but none of the SPACs had CUSIPs that matched both datasets.

2. FuzzyWuzzy -- We tried running a fuzzy match on the company names given in each dataset.
- This worked somewhat well but was prone to inaccurate readings.
- There was a company called 'Acquisition,' which the fuzzy match thought looked similar to any SPAC named 'Bob Joe Acquisition Corp.'
- When we deleted this company, we ran a fuzzy match that took 3 hours!
- It worked for about 60% of the data, but when combined with a confidence level of 90%, the fuzzy match ended up with only around 20% of SPACs.

3. Manually adding merge keys
- We realized that the most accurate way to gather all the data would be to merge on CUSIP, as all of the datasets have them, - and although some are different, we could manually go through the SPAC dataset to match them to the IPO and CCM datasets.
- This worked! We ended up creating CCM_Cusip.csv, which has CUSIP merge keys for both IPO data and CCM data.

#### Below is the final merge we came up with

In [6]:
# Downloading data into DFs
ipo_age_df = pd.read_csv('inputs/IPO-age(9).csv')
cleaned_spacs = pd.read_csv('inputs/CCM_Cusip.csv')
ccm_df = pd.read_csv('inputs/all_ccm_data.csv')

# Narrowing the horizons to be more memory-friendly
ipo_age_df = ipo_age_df[ipo_age_df['offer date'] > 20000000]
ipo_age_df = ipo_age_df[ipo_age_df['offer date'] < 20190000]
ipo_age_df = ipo_age_df.iloc[:, :-3]

# Merging the SPAC and IPO data
ziggymerge = pd.merge(ipo_age_df, cleaned_spacs, how='left', left_on='CUSIP', right_on='IPO_age_Cusip')

# Merging the previous merge with CCM Data
ziggymerge2 = pd.merge(left= ccm_df, right= ziggymerge, how='left', left_on='cusip', right_on='CCM_Cusip')
ziggymerge2.to_csv('inputs/masterMerge.csv')

/var/folders/x7/56zwk80s1dl703mqbltv_bq00000gn/T/ipykernel_68556/616900804.py:4: DtypeWarning: Columns (29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  ccm_df = pd.read_csv('inputs/all_ccm_data.csv')


We then filtered this data based on what was streamlit-friendly and what numeric values we thought would be best to model.

### Next up: Modeling!

In our final project proposal, we explored KNN and Logistic Regression models. We tested these two models and found that Logistic Regression had the best recall score for the dataset.

*Some caveats:*

1. Although Logistic Regression was the highest-scoring model, we achieved our highest score by using 66 variables! This is far too many for a user to input on our dashboard.

**How did we solve this?**

To address this, we conducted feature importance testing. We found that not many of the features had high correlations in determining whether a given IPO was a SPAC or not. However, we chose the top 5 variables to include on our dashboard. These variables were:

- Industry Code
- Cash Increase
- Market Value
- EPS (Earnings Per Share)
- Cash and Short Term Investments

2. We thought that, although it was less accurate, the KNN model was better for the interactive aspect of our dashboard. We believe this because users can see where their inputs lie within our model, and receive a 'Top 5 Most Comparable Companies' output.

**An issue with the KNN model:**

Since the dataset ranges from 2000 to 2018, there are many companies with multiple observations across the dataset. There is one company, WARNER CHILCOTT PLC, with 11 observations. Given that the SPAC dataset only covers from 2016 to 2021, and we narrowed this down to 2016 to 2018, the most observations a SPAC can have in this dataset is 3. This means that for a user input that the model guesses is most like a SPAC, there may be up to 3 of the top 5 comparable companies that are actually the same company in different years. We observed this with a couple of our tests of the dashboard.

### With this in mind, here are our models:

#### Logistic Regression Model

In [11]:
# Load the dataset
file_path = 'inputs/master_filtered_data.csv'
df = pd.read_csv(file_path)

# Drop duplicates
df.drop_duplicates(inplace=True)

# Get the total number of instances in the dataset
total_instances = len(df)

# Define features and the target variable
features = ['adrr', 'curuscn', 'scf', 'src', 'acominc',
                 'acox', 'at', 'am', 'ao', 'aoloch', 'aox', 'ap', 'at', 'caps', 'capx', 'cb',
                 'ch', 'che', 'clg', 'cogs', 'csho', 'cshrt', 'cstk', 'dd', 'dlc',
                 'dn', 'do', 'dt', 'ebit', 'ebitda', 'epspi', 'fca', 'ffo', 'gdwl',
                 'gp', 'ib', 'intan', 'invt', 'lt', 'lct', 'ni', 'niadj', 'np', 'pi', 'ppegt',
                 'pnrsho', 'ppent', 're', 'revt', 'sale', 'seq', 'tdc', 'teq', 'tstk', 'txt',
                 'wcap', 'naicsh', 'mkvalt', 'acchg', 'accrt', 'amc', 'ano', 'arce', 'cshi',
                 'depc', 'derhedgl']
target = 'IS_SPAC'

# Make sure the target column exists
if target not in df.columns:
    raise ValueError(f"Target column '{target}' not found in the dataset.")

# Fill NaN values in 'IS_SPAC' with 0 to indicate non-SPAC companies
df[target] = df[target].fillna(0)

# Convert the target column to an integer type
df[target] = df[target].astype(int)

# Convert categorical variables to numeric variables using one-hot encoding
df_clean = pd.get_dummies(df)

# Separate the features (X) and the target (y)
X = df_clean[features]
y = df_clean[target]

# Handle missing values in features by imputing with the mean of each column
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=10000)

model.fit(X_train, y_train)

# Make predictions on the test set using probability estimates
y_pred_proba = model.predict_proba(X_test)

# Define a range of thresholds
thresholds = np.linspace(0.15, 0.25, 10)

# Initialize variables to store the best F1 score and corresponding threshold
best_f1_score = 0
best_threshold = 0

# Find the threshold that maximizes the F1 score
for threshold in thresholds:
    y_pred_adjusted = (y_pred_proba[:, 1] >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred_adjusted)
    if f1 > best_f1_score:
        best_f1_score = f1
        best_threshold = threshold

# Use the best threshold to make predictions
y_pred_best_threshold = (y_pred_proba[:, 1] >= best_threshold).astype(int)

# Evaluate the model using a confusion matrix and metrics
conf_matrix = confusion_matrix(y_test, y_pred_best_threshold)
precision = precision_score(y_test, y_pred_best_threshold)
accuracy = accuracy_score(y_test, y_pred_best_threshold)
recall = recall_score(y_test, y_pred_best_threshold)

print("Best Threshold:", best_threshold)
print("Confusion Matrix:")
print(conf_matrix)
print("\nMetrics:")
print("Precision:", precision)
print("F1 Score:", best_f1_score)
print("Accuracy:", accuracy)
print("Recall:", recall)

Best Threshold: 0.18333333333333332
Confusion Matrix:
[[8277   31]
 [   6   23]]

Metrics:
Precision: 0.42592592592592593
F1 Score: 0.5542168674698795
Accuracy: 0.9955619527407941
Recall: 0.7931034482758621


For this model we also did threshold testing which can be seen in the image below:

![Threshold Tesing](outputs/F1ScoresVsThresholds.png)

#### KNN Model

In [8]:
# Load the data
merged_df = pd.read_csv('inputs/master_filtered_data.csv', low_memory=False)

# Define features and target column names
features = ['adrr', 'curuscn', 'scf', 'src', 'acominc', 'acox', 'at',
            'am', 'ao', 'aoloch', 'aox', 'ap', 'at.1', 'caps', 'capx', 'cb',
            'ch', 'che', 'clg', 'cogs', 'csho', 'cshrt', 'cstk', 'dd', 'dlc',
            'dn', 'do', 'dt', 'ebit', 'ebitda', 'epspi', 'fca', 'ffo',
            'gdwl', 'gp', 'ib', 'intan', 'invt', 'lt', 'lct', 'ni',
            'niadj', 'np', 'pi', 'ppegt', 'pnrsho', 'ppent', 're', 'revt',
            'sale', 'seq', 'tdc', 'teq', 'tstk', 'txt', 'wcap', 'naicsh',
            'mkvalt', 'acchg', 'accrt', 'amc', 'ano', 'arce', 'cshi',
            'depc', 'derhedgl']
target = 'IS_SPAC'

# Check which features exist in the dataset
valid_features = [col for col in features if col in merged_df.columns]

# Create a DataFrame with the required features and target
data = merged_df[valid_features + [target]]

# Handle missing values with mean imputation
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(data[valid_features])
y = data[target]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Instantiate and train KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = knn.predict(X_test_scaled)

# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 1.00
Precision: 0.72
Recall: 0.54
F1 Score: 0.62

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5534
         1.0       0.72      0.54      0.62        24

    accuracy                           1.00      5558
   macro avg       0.86      0.77      0.81      5558
weighted avg       1.00      1.00      1.00      5558


Confusion Matrix:
[[5529    5]
 [  11   13]]


### Finally, The Dashboard

The dashboard has been mentioned previously in our report, but there is more to explain. We built our dashboard in the "app.py" file, and used Streamlit to run it.

The dashboard provides a brief explanation of our project, our purpose, and then features two tabs for our models, along with a sidebar for user interaction with these models.

The KNN Model tab is made to be the more interactive of the two, where a user may input a company--real or fake--and our dashboard will indicate whether our model thinks the company is most likely a SPAC or an IPO. The dashboard also outputs the five most comparable company tickers.

The Logistic Regression Model is intended for useres who want to explore our model and data more closely. It displays the model performance through two different feature varaibles that the user selects on the sidebar drop down options. 

#### Screenshots of the dashboard showing the functionality of the Logistic Regression Model and the KNN Model:

![Sidebar](outputs/SideBar.png)
![KNN Model](outputs/KNNPLOT.png)
![Logistic Regression Model](outputs/LOGREGPLOT.png)